# Create a custom OBB table

hrsc2016-ms

`Chen, W.; Han, B.; Yang, Z.; Gao, X. MSSDet: Multi-Scale Ship-Detection Framework in Optical Remote-Sensing Images and New Benchmark. Remote Sens. 2022, 14, 5460. https://doi.org/10.3390/rs14215460`


## Project setup

In [2]:
PROJECT_NAME = "3LC Tutorials"
DATASET_NAME = "HRSC2016-MS"

## Imports

In [3]:
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict
import xml.etree.ElementTree as ET
import tlc

## Prepare the data

https://github.com/wmchen/HRSC2016-MS


In [4]:
DATASET_ROOT = "D:/Data/HRSC2016-MS"

tlc.register_url_alias("HRSC2016_MS_DATA", DATASET_ROOT)

In [5]:
row_data = defaultdict(dict)

for split in ["train", "val"]:
    image_splits = Path(DATASET_ROOT) / "ImageSets" / f"{split}.txt"
    image_ids = image_splits.read_text().splitlines()
    row_data[split] = defaultdict(list)

    for image_id in image_ids:
        image_path = Path(DATASET_ROOT) / "AllImages" / f"{image_id}.bmp"
        annotation_path = Path(DATASET_ROOT) / "Annotations" / f"{image_id}.xml"
        if not image_path.exists():
            print(f"Image {image_id} does not exist")
        if not annotation_path.exists():
            print(f"Annotation {image_id} does not exist")
        row_data[split]["image"].append(tlc.Url(image_path).to_relative().to_str())
        row_data[split]["obb"].append(annotation_path)

In [ ]:
def load_obb_annotation(annotation_path):
    """Example annotation:
    """
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)
    annotations = {
        "instances": [],
        "instances_additional_data": {
            "label": [],
            "difficult": [],
            "truncated": [],
        },
        "x_max": width,
        "y_max": height,
    }
    for obj in root.findall("object"):
        name = int(obj.find("name").text)
        difficult = int(obj.find("difficult").text)
        truncated = int(obj.find("truncated").text)
        bbox = obj.find("robndbox")
        cx = float(bbox.find("cx").text)
        cy = float(bbox.find("cy").text)
        w = float(bbox.find("w").text)
        h = float(bbox.find("h").text)
        angle = float(bbox.find("angle").text)

        annotations["instances"].append(
            {
                "oriented_bbs_2d": [
                    {
                        "center_x": cx,
                        "center_y": cy,
                        "size_x": w,
                        "size_y": h,
                        "rotation": angle,
                    }
                ]
            }
        )

        annotations["instances_additional_data"]["label"].append(name)
        annotations["instances_additional_data"]["difficult"].append(bool(difficult))
        annotations["instances_additional_data"]["truncated"].append(bool(truncated))


    return annotations

In [9]:
for split in ["train", "val"]:
    row_data[split]["obb"] = [load_obb_annotation(path) for path in row_data[split]["obb"]]

In [10]:
schemas = {
    "image": tlc.Schema(value=tlc.ImageUrlStringValue()),
    "obb": tlc.Geometry2DSchema(
        include_2d_oriented_bounding_boxes=True,
        per_instance_schemas={
            "label": tlc.CategoricalLabelListSchema(["ship"]),
            "difficult": tlc.BoolListSchema(),
            "truncated": tlc.BoolListSchema(),
        }
    )
}

In [11]:
for split in ["train", "val"]:
    table = tlc.Table.from_dict(
        data=row_data[split],
        structure=schemas,
        table_name=f"{split}",
        dataset_name=DATASET_NAME,
        project_name=PROJECT_NAME,
        if_exists="rename",
    )